<a href="https://colab.research.google.com/github/RonildoSilva/datasets/blob/main/big_black_forecast_da_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime

! pip install flood-forecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import wget
import os.path

if(not os.path.exists('01010500FVE_flow.csv.zip')):
  wget.download('https://www.kaggleusercontent.com/kf/102610075/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..TS68NOi_pMM4hMNth5c69g.GU1e3tDnb-d0Mn9QQ4Fp5npgGhAWy2q8XGplg843NF0Nl0pBAIv4vqrnkqRS6Y7f2B1nll38OxeocRnFUjqYcQ1P80HxWqCkizLnWPlDryE-SdsRI7vaK21Y1oFEFI-_trZ5Bf0x7e899FcV_i_5Fd9zQ7WYvxuTM6a-qm3OboSFKYBV4hkyEQTwmJG0sOhdzgTRbzBBAirqflChrjK1KhLsVb5BQiD3LStbB-v8kDuAmZAVwXUC5ekrWsTxDJiIbRawLNpUEgV-9S4Ciw30r4eQjSK6Cr_mbNo719gprwdyFG2_nPcRBYr6YmgihWtMI9n9bUHTHbnGBo-Sz47LFS_epohe1S6otOnDxEGzC0moUw9gl4s2MFuBeZrFa5hIHjWffp3B8euaulcqMH-nZ6dLvq8RyrfHcFKScDQ1UZvfqAHIh8Pl_4CSJJ4SkTOqID2inybkUIZO4J4aKLFY3g3KlVriiBBZ5-CQJFOn9D03EzmDfID5LtpTCH8Vg2s2tblUGP2cWa47Grn3FvrXdza5j0TOeFd4wXKIgn_BubMceX6TnIEOoKE70z58vqA-FCH1OJKXdP2jKqpk_iMy3_JeQ54pW8ROO3pLTJXt_3mai5yIZu31xrvYPv9TbHKyY1XtOj1Eqmd8TsS9vm42NdCSfw9hoDqvpwbmiich6EQqZzCt_dqC9UyyqonU9Y4T.bifvCHXXZbBtlyHw_0TEPw/flow-forecast/01010500FVE_flow.csv')

In [4]:
#! unzip 01010500FVE_flow.csv.zip

In [5]:
import pandas as pd

df = pd.read_csv("01010500FVE_flow.csv")

df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("01010500FVE_flow.csv")

df[["hour_updated", "cfs", "tmpf", "p01m", "dwpf"]].head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,hour_updated,cfs,tmpf,p01m,dwpf
0,2000-04-04 04:00:00+00:00,21700.0,42.08,0.0,26.96
1,2000-04-04 05:00:00+00:00,21800.0,41.00,0.0,26.96
2,2000-04-04 06:00:00+00:00,22000.0,39.02,0.0,26.96
3,2000-04-04 07:00:00+00:00,22000.0,39.02,0.0,28.94
4,2000-04-04 08:00:00+00:00,22100.0,37.94,0.0,30.02


In [23]:
def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
    run = wandb.init(project="github_aistream-peelout_flow-forecast")
    wandb_config = run.config
    the_wandb_c = run.config
    print(wandb_config)
    the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series": 4,
        "forecast_history":wandb_config["forecast_history"],
        "hidden_size_encoder":wandb_config["hidden_encoder"],
        "decoder_hidden_size": wandb_config["hidden_decoder"],
        "out_feats": 1,
        "dropout": wandb_config["dropout"],
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":12,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"] - 1,
       "forecast_length":wandb_config["forecast_length"],
       "scaler": "StandardScaler",
       "train_start":0,#1000,
       "train_end": 100,#50000,
       "valid_start":101,#50001,
       "valid_end": 151,#57000,
       "sort_column": "datetime",# "hour_updated",
       "test_start": 151,#57000,
       "test_end":200,#58000,
       "target_col": ["TimeConclusion"],
       "relevant_cols": ["TimeConclusion", "Duration"], 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["TimeConclusion", "Duration"]}
           },
      "feature_param":
          {
            "datetime_params":{
            "hour":"numerical",
            "month": "numerical"
            }
          }
       },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": the_wandb_c["lr"]
    },
       "epochs": 12,
       "batch_size":wandb_config["batch_size"]
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "MAPE"],
   "inference_params":
   {     
          "datetime_start":"2010-01-13",#"2020-05-31",
          "hours_to_forecast":336, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "datetime", #"hour_updated",
             "scaling": "StandardScaler",
             "forecast_history": wandb_config["forecast_history"] - 1,
             "forecast_length":wandb_config["forecast_length"],
             "relevant_cols": ["TimeConclusion", "Duration"],
             "target_col": ["TimeConclusion"],
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":["TimeConclusion", "Duration"]}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "hour":"numerical",
            "month": "numerical"
         
     }
             }
          }
          } 
    }

      
    if weight_path:
        the_config4["weight_path"] = weight_path
    wandb.config.update(the_config4)
    print("config made")
    return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]}
            ,
        "batch_size": {
            "values": [200]#, 400, 600]
        },
        "lr":{
            "values":[0.001]#, 0.0001, .01]
        },
        "forecast_history":{
            "values":[14]#, 20, 24, 48]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32]#, 64, 128, 256]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[32]#, 64, 128, 256]
        }, 
        "dropout":
        {
            "values": [0.1],#, 0.3, 0.5, 0.8]
        }
  
    }
}

In [7]:
import wget
import os.path

if(not os.path.exists('df_train_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_train_keras_tk.csv')

if(not os.path.exists('df_val_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_val_keras_tk.csv')

if(not os.path.exists('df_test_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_test_keras_tk.csv')

In [8]:
import pandas as pd 
df = pd.read_csv('df_train_keras_tk.csv')#.loc[151]['datetime']

In [9]:
pd.read_csv('df_train_keras_tk.csv').loc[151]['datetime']

'2013-06-25'

In [10]:
#df['datetime']

In [19]:
#df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
#2000-04-04 04:00:00+00:00

In [20]:
df = df[["TimeConclusion", "Duration", "datetime"]]
df.to_csv('df_train_keras_tk.csv', index=False)

In [21]:
df.head()

,TimeConclusion,Duration,datetime
0,31.0087,0.0000,2012-10-09 00:00:00+00:00
1,15.0003,16.0084,2012-10-09 00:00:00+00:00
2,15.0002,0.0001,2012-10-12 00:00:00+00:00
3,0.0000,15.0002,2012-10-25 00:00:00+00:00
4,30.9822,0.0000,2012-11-09 00:00:00+00:00


In [13]:
"""
!pip install -Uqq ipdb
import ipdb
%pdb on
"""

'\n!pip install -Uqq ipdb\nimport ipdb\n%pdb on\n'

In [14]:
df.columns

Index(['TimeConclusion', 'Duration', 'datetime'], dtype='object')

In [24]:
import wandb
from flood_forecast.trainer import train_function


sweep_id = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
#sweep_id = "03l75ilz"

#file_path = "01010500FVE_flow.csv"
file_path = "df_train_keras_tk.csv"

wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None)))

#model = train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None))

Create sweep with ID: 1djgsnaf
Sweep URL: https://wandb.ai/ronildosilva/github_aistream-peelout_flow-forecast/sweeps/1djgsnaf


wandb: Agent Starting Run: ndinusi2 with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 32
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.001, 'out_seq_length': 1}
config made
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['hour', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['hour', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['hour', 'month']
scaling now
Using Wandb config:
{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.001, 'out_seq_length': 1, 'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 4, 'forecast_history': 14, 'hidden_size_encoder': 32, 'decoder_hidden_size': 32, 'out_feats': 1, 'dropout': 0.1, 'gru_lstm': False}, 'dataset_params': {'class': 

This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


The running loss is: 
1.0518139600753784
The number of items in train is: 1
The loss for epoch 0
1.0518139600753784
Computing validation loss
running torch_single_train


...

test loss: 29.014607475604862
This model is currently forecasting for: 1 targets
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['hour', 'month']
scaling now
CSV Path below
df_train_keras_tk.csv
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
<class 'numpy.ndarray'>
Un-transforming data
Current historical dataframe 
      TimeConclusion  Duration   datetime  original_index  hour  month  \
1851         34.9812    0.0245 2012-01-18            1851     0      1   

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Using a non-full backward hook when outputs are generated by different autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_output. Please use register_full_backward_hook to get the documented behavior.
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1405.)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input

plotting with CI now
Now plotting final plots


average_prediction_sharpe0,▁
epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
loss,▃██▆▃▄█▆▁▄▅▃
average_prediction_sharpe0,1.12104
epoch,11
loss,1.0515


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
